<h1 align="center">Introduction to Amazon SageMaker AI</h1>

## Table of Contents
1. [Overview](#Overview)
2. [Amazon S3](#Amazon-S3)
   * [Introduction](#Introduction)
   * [Creating bucket](#Creating-bucket)
   * [Uploading data](#Uploading-data)
3. [Amazon SageMaker AI](#Amazon-SageMaker-AI)
   * [Processing jobs](#Processing-jobs)
   * [Training jobs](#Training-jobs)
   * [Endpoints](#Endpoints)
   * [Batch transform jobs](#Batch-transform-jobs)
4. [CloudWatch Logs](#CloudWatch-Logs)
5. [Epilogue](#Epilogue)

## Overview
Amazon SageMaker AI is an umbrella of services that AWS provides for Machine Learning (ML). In a nutshell, it is a service that enables the developer to be much more efficient with their valuable time when developing and deploying ML models. This methodology is applicable across many algorithms and many production use cases.

In this tutorial, you will harness some of the most commonly used microservices of SageMaker AI to contruct basic components of a machine learning workflow. By the end of this tutorial, you will be able to:
* Launch a processing job to preprocess your data.
* Launch a training job and build your ML model.
* Deploy an endpoint to serve as an API for your trained model.
* Launch a batch transform job to try out your trained model.
<center><img src="img/sagemaker_microservices.png" width="80%"></center>

## Amazon S3
First of all, you need to create a bucket in Amazon S3 to store any future files and data.

### Introduction

Amazon Simple Storage Service (Amazon S3) is an object storage service that can store almost any object needed for machine learning. That includes datasets, model artifacts, logs, and more.

An [S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html) is a container for objects (i.e., files) stored in S3. 

S3 supports the folder concept as a means of grouping objects. It does this by using a shared name *prefix*. In other words, the grouped objects have names that begin with a common string. This common string, or shared prefix, is the folder name. The prefix must end with a forward slash character `/` to indicate folder structure. Furthermore, object names are also referred to as key names.

For example: `s3://example-bucket/1/2/3/example.txt`
 * Bucket: `example-bucket`
 * Prefix: `1/2/3/`
 * Key name: `1/2/3/example.txt`

### Creating bucket

First step is importing `boto3` module, which is the AWS SDK for Python. You are encouraged to explore [its documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for future practice.

In [ ]:
import boto3
from botocore.exceptions import ClientError

You will then connect to Amazon S3 and create a bucket. Enter a name for your bucket in the code below. It must be globally unique accross all AWS accounts. Once created, you cannot change its name.

In [ ]:
# Create a service client to access S3
s3 = boto3.client('s3')

bucket_name = '?'  # Replace with a globally unique name for your bucket
s3.create_bucket(Bucket=bucket_name)

Now that you have created a bucket on AWS, you can upload any files into this storage through `boto3` or AWS console. Furthermore, you can create folders to help organize your files more effectively. Simply run the code below and feel free to change the names, but __do not remove__ the trailing slash character `/` of the prefixes. You can also create nested folders by including more slashes, such as `parent-folder/child-folder/`.

In [ ]:
data_prefix = 'data/'  # Folder for datasets
model_prefix = 'models/'  # Folder for models

s3.put_object(Bucket=bucket_name, Key=data_prefix)
s3.put_object(Bucket=bucket_name, Key=model_prefix)

### Uploading data

After creating bucket, you can upload data and any other files there. But right now, you need to upload the dataset [`reviews_Toys_and_Games_5.json.zip`](../data/reviews_Toys_and_Games_5.json.zip).

In [ ]:
def upload_file_to_s3(filename, s3_prefix=''):
    key_name = s3_prefix + filename.rsplit('/', maxsplit=1)[-1]
    try:
        s3.upload_file(filename, bucket_name, key_name)
    except ClientError as e:
        print(e)

In [ ]:
upload_file_to_s3('../data/reviews_Musical_Instruments_5.json.zip', data_prefix)

If done successfully, you can see the dataset in your bucket from [S3 console](https://console.aws.amazon.com/s3/home).

## Amazon SageMaker AI

After uploading necessary files, the next thing to do is training a machine learning model and making use of it to produce inferences. This is the most important part as you will perform common machine learning operations on AWS.

Step by step, you will create a model that predicts the usefulness of a product review, given only the text. This is an example of a problem in the domain of supervised sentiment analysis.

In the upcoming steps, you are going to use `sagemaker` module, which is a higher-level AWS SDK specifically designed for SageMaker AI. Although `boto3` gives you general access to all AWS services, this module is specialized for tasks within the SageMaker AI microservices. You are also going to use the role that was created for this notebook instance and its session to run SageMaker AI microservices.

In [ ]:
import sagemaker

# Get the execution role
role = sagemaker.get_execution_role()
# Get current session
session = sagemaker.Session()

### Processing jobs

Before training a model, you need input data. The [dataset](../data/reviews_Toys_and_Games_5.json.zip) you will be working with is a collection of reviews for an assortment of toys and games found on Amazon. It includes, but is not limited to, the text of the review itself as well as the number of user votes on whether or not the review was helpful.

However, the dataset is inside a .zip file so you have to extract it before proceeding. Moreover, the dataset is a file containing a single JSON object per line representing a review with the following format: 
```JSON
{
 "reviewerID": "<string>",
 "asin": "<string>",
 "reviewerName": "<string>",
 "helpful": [
   <int>, (indicating number of helpful votes)
   <int>  (indicating total number of votes)
 ],
 "reviewText": "<string>",
 "overall": <int>,
 "summary": "<string>",
 "unixReviewTime": <int>,
 "reviewTime": "<string>"
}
```

Later, you will be using [BlazingText algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html) in training process, which is an implemention of [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) optimized for SageMaker AI. Therefore, in order for this algorithm to work, you have to format the input data correctly. This is true for any other algorithm or model you work with, as each of them requires a particular type and structure of the input data. In this case, the data should only consist of plain text, with each line containing a label name followed by a sentence. Labels must be prefixed by the string `__label__`. 

For the dataset in this exercise, you will extract the text from the field *reviewText* and generate label based on the field *helpful* for each review. If the majority of votes is helpful, assign it `__label__1`, otherwise `__label__2`. If there is no majority or the review text is empty, drop the review from consideration. Then, cut the text into individual sentences, while ensuring that each sentence retains the original label from the review. When splitting using the character `.`, make sure that no empty sentences are created, since reviews usually contain an ellipsis `...` or more. Your input data should look something like this:
```
__label__1 Even if you can only play with one other person, you'll want to pull Stone Age out often
__label__1 But if you have friends to join you, this game will be on the table a lot
__label__2 It's a fun game but not a favorite
__label__2 I prefer more complex games
__label__2 If you're new to gaming or like relatively simple games I recommend you try this
```

Finally, it is your responsibility is to split the dataset into training set and testing set. Training set should represent 80% of the dataset, while the rest is testing set. Make sure that they don't overlap.

All of the procedures mentioned above and more are collectively called *data pre-processing*, the first and most crucial step in any machine learning project.

Now, implement a Python script to unzip, format, and split the raw dataset as previously instructed and save it in the same folder as this notebook. Or you can go ahead and use [hello_blaze_preprocess.py](hello_blaze_preprocess.py) provided. After that, upload the script to your S3 bucket.

If you decide to custom your own script, note that the processing job will copy the dataset from S3 to a local directory, prefixed with `/opt/ml/processing/`, within the container. Thus, your script should take the dataset in this directory as input instead. Additionally, it must output training set and testing set to specified local directories, also prefixed with `/opt/ml/processing/`. You will have a chance to set up these local directories in the next steps.
<center><img src="img/processing_model.png"></center>

In [ ]:
# Replace with your Python script
preprocess_code = '?'
upload_file_to_s3(preprocess_code, data_prefix)

Before creating a processing job, you have to set up a few things:
* A unique name for your processing job in `job_name`. If not specified, it will be auto-generated.
* The local path that the dataset will be downloaded into, e.g. `/opt/ml/processing/input/data`, and the S3 location (S3 URI) of the dataset.
* The S3 location of your Python script.
* The local path where your Python script saves the training set and testing set and the S3 location where you want SageMaker AI to upload them.
<center><img src="img/s3_location.png"></center>
    
If you use the provided code, you need to set the name of the dataset as a container argument. Otherwise, set `arguments` to `None`. Under the hood, the processor will execute a command line to run your script within its container. Something like this:
```
python3 /opt/ml/processing/input/code/hello_blaze_preprocess.py review_Toys_and_Games.json.zip
```

In [ ]:
dataset_name = 'reviews_Toys_and_Games_5.json.zip'
container_prefix = '/opt/ml/processing/'
job_name = None
arguments = [dataset_name]

# Local directory path that the dataset will be downloaded into
input_path = container_prefix + 'input/data'
# Local directory paths where your Python script saves the training/testing set
train_path = container_prefix + 'output/train'
test_path = container_prefix + 'output/test'

print("Raw dataset will be downloaded to", input_path)
print("Python script saves training set in", train_path)
print("Python script saves testing set in", test_path)

In [ ]:
def get_s3_path(filename='', prefix=''):
    if prefix != '' and prefix[-1] != '/':
        raise ValueError("Prefix must have trailing forward slash character.")
    return 's3://' + bucket_name + '/' + prefix + filename

In [ ]:
# S3 location of your preprocessing script
code_s3 = get_s3_path(filename=preprocess_code, prefix=data_prefix)
# S3 location of the unprocessed dataset
dataset_s3 = ?

# S3 path where you want SageMaker AI to upload the training/testing set
train_s3 = get_s3_path(prefix=data_prefix+'train/')
test_s3 = ?

print("S3 location of preprocessing script is", code_s3)
print("S3 location of unprocessed dataset is", dataset_s3)
print("SageMaker AI will upload the training set to", train_s3)
print("SageMaker AI will upload the testing set to", test_s3)

After configuring any neccessary parameters, you will begin to run a processing job with them. For this exercise, you will utilize [scikit-learn processor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) and its container to run the job.

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Create an SKLearnProcessor, version 1.2-1
sklearn_processor = SKLearnProcessor(framework_version='1.2-1',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1,
                                     sagemaker_session=session)

# Start a processing job.
sklearn_processor.run(code=code_s3,
                      inputs=[ProcessingInput(source=dataset_s3, destination=input_path)],
                      outputs=[ProcessingOutput(source=train_path, destination=train_s3),
                               ProcessingOutput(source=test_path, destination=test_s3)],
                      arguments=arguments,
                      job_name=job_name)

Wait for a few minutes to see if your processing job executed successfully or not. It it failed, investigate the cause of failure from the logs generated by the code above. Once the issue is identified and fixed, try to run the code again. If it succeeded, you should be able to see the training set and testing set in your S3 bucket.

Your input data is now ready for use in training. Let's move to the next section.

### Training jobs

A training job in SageMaker AI is a managed execution of training a machine learning algorithm. You simply provide the dataset and configure the training job with the necessary parameters. SageMaker AI will take care of running the training process. When the job has completed, the resulting model artifact will be stored in the S3 location you specified.

A model artifact is a package generated by a training job. This packaged model contains the information of parameters learned from the training process and instructions on how to effectively perform predictions, known as *inference*.

After you successfully pre-processed your dataset, it is time to train a model with it. As mentioned earlier, you will use the BlazingText algorithm provided by SageMaker AI as your model. Therefore, you need to retrieve the location where container image of this Amazon built-in algorithm is stored.

In [ ]:
# Get the container location of BlazingText
container = sagemaker.image_uris.retrieve(framework='blazingtext',
                                          region=session.boto_region_name,
                                          version='latest')
container

Then, start configuring your training job and set up some hyperparameters for the model. In this exercise, your training job will use:
* BlazingText's container image.
* One count of the instance `ml.m5.large` with 5GB of additional memory.
* Maximum runtime of 10 minutes. It is important that you set runtime limit for your training job because if it takes a long time, you will be charged more.
* Supervised mode of the algorithm with default hyperparameters.

In [ ]:
# S3 path where you want to save model artifact
model_s3 = ?
print("Your model artifact will be save in", model_s3)

# Configure the training job
model = sagemaker.estimator.Estimator(image_uri=container,
                                      role=role,
                                      sagemaker_session=session,
                                      instance_count=?,
                                      instance_type=?,
                                      volume_size=5,
                                      max_run=?,  # 10 minutes in seconds
                                      output_path=?,
                                      debugger_hook_config=False,
                                      disable_profiler=True)

# Set hyperparameters
model.set_hyperparameters(mode='supervised')

Now that your training job has been set up, simply attach the S3 location of the training set and testing set to it and run the job. Since the chosen algorithm only works with one type of data, plain text, you don't have to specify the content type of your data. But if you work with other algorithms, pay attention to this in their documentation, as they may have different data format options to select. Moreover, you can give a unique name to the job or let it have auto-generated name.

In [ ]:
model.fit(inputs={'train': ?,
                  'validation': ?},
          job_name=None)

The training job should complete within 10 minutes and then you will be able to find the model artifact generated by the job in your S3 bucket. The model is being compressed inside a .tar.gz file. If your training job fails, it is mostly due to incorrectly formatted input data or misconfiguration in the setup.

Exciting! Your model is now ready to deploy and use in production.

### Endpoints

In SageMaker AI, an endpoint serves as an Application Programming Interface (API) to a deployed model in production. It provides a [RESTful interface](https://aws.amazon.com/what-is/restful-api/) that allows you to send user data to the model and receive predictions in response, enabling real-time inference.

To deploy your trained model with an endpoint, you can directly use the instance of `Estimator` created previously. This will automatically create a model object and an endpoint configuration in SageMaker AI. Once the endpoint is ready to use, you can then interact with it and make real-time predictions on new data. 

In [ ]:
endpoint_1 = model.deploy(initial_instance_count=1,
                          instance_type='ml.m5.large',
                          endpoint_name=None)
print("\nEndpoint #1 name:", endpoint_1.endpoint_name)

Alternatively, you can create a model object explicitly from a model artifact and deploy it to an endpoint. This is useful if you want to use an existing model in your bucket, possibly with different configurations.

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

# Create a model object
new_model = Model(image_uri=container,
                  model_data=model.model_data,  # Location of model artifact
                  role=role,
                  sagemaker_session=session)
# Deploy to an endpoint
new_model.deploy(initial_instance_count=1,
                 instance_type='ml.m5.large',
                 endpoint_name=None)
print("\nEndpoint #2 name:", new_model.endpoint_name)

endpoint_2 = Predictor(new_model.endpoint_name, sagemaker_session=session)

When sending a prediction request to your endpoint, check if the input data adheres to the inference data format accepted by the algorithm you are using. It is highly recommended to consult the documentation of your select algorithm for the data format requirements and other important information. 

For BlazingText algorithm, the inference data requires a different structure from the training data, as found in [its documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). You need to send a prediction request in the form of JSON object and indicate the content type of the request, which is `application/json`.


In [ ]:
import json
# Replace with your custom review sentences
request = json.dumps({'instances': ['Review sentence #1',
                                    'Review sentence #2']})

print("Prediction from endpoint #1:")
print(endpoint_1.predict(request, initial_args={'ContentType': 'application/json'}))

print("Prediction from endpoint #2:")
print(endpoint_2.predict(request, initial_args={'ContentType': 'application/json'}))

<ins>**Please note**</ins>: Creating an endpoint and keeping it active **does incur costs** on your AWS account, even if it's idle. To avoid unwanted charges, delete any endpoint right after you finish your experiment, or when you are stepping away.

In [ ]:
endpoint_1.delete_predictor()
endpoint_2.delete_predictor()

### Batch transform jobs

Batch Transform is a compute job that performs inference using your trained model on large datasets. A batch transform job is ideal for large-scale predictions where low latency is not a priority. This differs from an endpoint in that endpoints service real-time, individual user requests for inference over time.

To deploy a model using batch transform, there are also two approaches similar to the previous section. But first, you need inference data. Create a .json file to be your inference data and write down a list of custom review sentences, as many as you like. Then, upload this file to your S3 bucket and retrieve its S3 location.

In [ ]:
# Replace with your custom review sentences
inference_data = '''{
  "instances": [
    "Review sentence #1.",
    "Review sentence #2.",
    "Review sentence #3.",
    "Review sentence #4.",
    "...",
  ]
}
'''
inference_filename = 'custom_reviews.json'

with open(inference_filename, 'w') as f:
    f.write(inference_data)
upload_file_to_s3(inference_filename, 'inference/')
inference_loc = ?

For the configuration of the batch transform job, set `split_type` and `assemble_with` to `'Line'`. This tells SageMaker AI to process the input file line by line, which can paralellize the job and improve performance, and assemble the predictions into a single output file. Furthermore, set `content_type='application/json'`, as this is the required data format for inference by BlazingText algorithm. Another acceptable format is `application/jsonlines`. If you are using a trained model from a different algorithm, be sure to check and specify correctly the required input format for inference in its documentation.

In [ ]:
batch_job = model.transformer(instance_count=1,
                              instance_type='ml.m5.large',
                              assemble_with=?,
                              output_path=?)
batch_job.transform(data=inference_loc,
                    content_type=?,
                    split_type=?,
                    job_name=None)

Within a few minutes, your batch transform job will be done and you can find the result in your S3 bucket. It's an .out file containing a label prediction associated with a probability for each review sentence. Open it as a text file and you will see. If the job failed, please double-check its configuration and the format of your inference data.

You may notice how poorly the model performs text classification. Since this tutorial only focuses on using microservices of SageMaker AI, it does not go over any tips on how to improve your model. For that, you may need to train a new model with fine-tuned [hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html#blazingtext_hyperparameters_text_class) or with a different algorithm. Finally, always read the documentation of the algorithm you are working with to minimize error and understand its hyperparameters. For a list of built-in algorithms of AWS and their documentation, click [here](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html).

## CloudWatch Logs

CloudWatch Logs is the log repository of your compute resources in AWS, including those used by SageMaker AI. In practice, configuring and managing in cloud services is difficult and error-prone. This is where CloudWatch Logs comes in handy. It helps you identify and troubleshoot the cause of error in a AWS service.

Log into your AWS Console and navigate to CloudWatch through the search bar. After that, go to *Logs* → *Log groups* and you can see log group of every AWS service you've worked with so far. You can diagnose issues of a SageMaker AI job from there.
<center><img src="img/cloudwatch.png"></center>

Alternatively, you can go to the detail of a failed job and click on *View logs* from the section *Monitoring* or *Monitor*. This takes you to the relevant log stream in CloudWatch, allowing you to quickly inspect error messages.
<center><img src="img/view_logs.png" width="60%"></center>
<center><img src="img/error_log.png"></center>

## Epilogue

Congratulations! You have successfully completed the first milestone. 

As a best practice, __remember to delete or turn off__ any cost-consuming instances in AWS SageMaker AI. Moreover, you are encouraged to practice what you have learned so far to a different use case.

Throughout a machine learning model development, creating a seperate job in SageMaker AI for each phase is tiresome and inefficient. Fortunately, AWS provides a way to automate and orchestrate these steps into a single, streamlined workflow. Let's jump to the next tutorial and find out.